In [1]:
import pandas as pd
import numpy as np

In [41]:
from scipy.optimize import minimize
import gurobipy as gp

In [14]:
def create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE, ub=None, lb=None, vtype=None, time_limit=3600, Q=None): 

    # creating model
    model = gp.Model()
    
    if vtype is None:
        vtype = ['C'] * A.shape[1]

    # creating variable an setting the constraints
    if (ub is None) & (lb is None):
        modx = model.addMVar(A.shape[1], vtype=vtype)
    elif (ub is not None) & (lb is None):
        modx = model.addMVar(A.shape[1], ub=ub, vtype=vtype)
    elif (ub is None) & (lb is not None):
        modx = model.addMVar(A.shape[1], lb=lb, vtype=vtype)
    else:
        modx = model.addMVar(A.shape[1], lb=lb, ub=ub, vtype=vtype)
        
    mod_con = model.addMConstrs(A, modx, sense, b)

    # setting the objective function
    model.setMObjective(Q, obj, 0, sense=opt)
 
    # restricting gurobi logs
    model.Params.OutputFlag = 0
    model.setParam('TimeLimit', time_limit)

    # optimizing the function
    model.optimize()
    
    return model

## Integer programming

Ferrari is planning to design a new car.  They have 3 engines to choose from to put in the car, engine 1, 2 or 3.  Engine 1 makes 450 horsepower (hp) and costs usd50000, engine 2 makes 600 hp and costs usd55000, and engine 3 makes 750 hp and costs usd65000.  Ferrari can also add a turbo charger to engines 1 or 2 if they want to, but not to engine 3.  A turbo charger would add 200 horsepower to a engine 1 or 150 hp to engine 2, and costs usd5000 on either engine.  Ferrari must also choose between 2 sets of breaks, beaks 1 or 2.  Breaks 1 cost usd5000 and breaks 2 cost usd7000. If a turbo charger is added to the engine, then Ferrari must use breaks 2.  They also have 2 interior options, interior 1 or 2.  Interior 1 costs usd15000 and interior 2 costs usd20000.  Ferrari knows that they can only sell cars with interior 1 if the car uses engine 3 or a has a turbo charger.  The car must pick exactly 1 engine, exactly 1 set of breaks and exactly 1 interior.  The car may or may not have a turbo charger; if it has a turbo charger it can only have 1.  Ferrari wants to design the least expensive car that has at least 600 hp.

 

Let e1, e2, e3 be the binary variables representing which engine to put in the car.

Let t1 and t2 be the binary variables representing whether a turbo charger is installed on engines 1 or 2.

Let b1 and b2 be the binary variables representing which set of breaks get put on the car.

Let i1 and i2 be the binary variables representing which interior is used in the car.

 

How much does it cost to build the optimal car? Round to the nearest dollar

 

Hint: The way I solved the problem there are 9 constraints

1) You must pick only 1 engine

2) Turbo1 can only be used on engine1

3) Turbo2 can only be used on engine2

4) Only one of the turbos can be used (this might be redundant?)

5) You must pick only 1 set of breaks

6) Breaks2 must be used if you use a turbo

7) You must pick only 1 interior

8) Interior 1 can only be used if you use engine 3 or a turbo

9) The car must have at least 600 hp

In [23]:
n_const = 9
n_var = 9
A = np.zeros((n_const, n_var))
b = np.zeros(n_const)
sense = ['<']*n_const

obj = np.array([50000, 55000, 65000, 5000, 5000, 5000, 7000, 15000, 20000])

In [24]:
A[0, 0:3] = 1
b[0] = 1
sense[0] = '='

In [25]:
A[1, [0, 3]] = [-1, 1]

In [26]:
A[2, [1, 4]] = [-1, 1]

In [27]:
A[3, [3, 4]] = 1
b[3] = 1

In [28]:
A[4, 5:7] = 1
b[4] = 1
sense[4] = '='

In [29]:
A[5, [3, 4, 6]] = [1, 1, -1]

In [30]:
A[6, -2:] = 1
b[6] = 1
sense[6] = '='

In [31]:
A[7, [2, 3, 4, 7]] = [-1, -1, -1, 1]

In [32]:
A[8, :5] = [450, 600, 750, 200, 150]
b[8] = 600
sense[8] = '>'

In [33]:
ferari = create_model(A, sense, b, obj, opt=gp.GRB.MINIMIZE, vtype=['B']*n_var)

<ipython-input-33-44d93b541e72>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  ferari = create_model(A, sense, b, obj, opt=gp.GRB.MINIMIZE, vtype=['B']*n_var)


In [34]:
ferari.objVal

77000.0

In [35]:
ferari.x

[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]

## Non-Linear programming

In [104]:
def electricity_diff(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    
    e1 = 50*x1
    e2 = 50*x2 + 20*x1*x1
    e3 = 50*x3 + 20*x2*x2 + 10*x1*x1
    
    supply = np.array([e1/3 + e2/3,  e1/3 + e3/3, e2/3 + e3/3, e1/3 + e2/3 + e3/3])
    demand = np.array([44, 20, 50, 18])
    
    sse = np.sum((supply - demand)**2)
    
    return sse

def min_demand_const(x):
    
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    
    e1 = 50*x1
    e2 = 50*x2 + 20*x1*x1
    e3 = 50*x3 + 20*x2*x2 + 10*x1*x1
    
    supply = np.array([e1/3 + e2/3,  e1/3 + e3/3, e2/3 + e3/3, e1/3 + e2/3 + e3/3])
    demand = np.array([44, 20, 50, 18])
    
    return np.sum(supply) - np.sum(demand)
  
def get_supply(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    
    e1 = 50*x1
    e2 = 50*x2 + 20*x1*x1
    e3 = 50*x3 + 20*x2*x2 + 10*x1*x1
    
    supply = np.array([e1/3 + e2/3,  e1/3 + e3/3, e2/3 + e3/3, e1/3 + e2/3 + e3/3])
    
    return supply

In [56]:
con1 = {'type':'ineq', 'fun': min_demand_const}
cons = [con1]
bds = [(0,1),(0,1),(0,1)]
w = np.ones(3)*0.9

In [57]:
opt_port = minimize(electricity_diff, w ,constraints=cons,bounds=bds) 

In [58]:
opt_port.x

array([0.7752806 , 1.        , 0.10408339])

In [59]:
opt_port.fun

1153.3437087898453

# Exam

### q1

In [74]:
n_var = 9
n_const = 8

A = np.zeros((n_const, n_var))
b = np.zeros(n_const)
sense = ['<']*n_const

In [85]:
A[0, :3] = 1
b[0] = 1
sense[0] = '='

In [86]:
A[1, [0, 3]] = [-1, 1]

In [87]:
A[2, [1, 4]] = [-1, 1]

In [88]:
A[3, [5, 6]] = 1
b[3] = 1
sense[3] = '='

In [89]:
A[4, [3, 4, 6]] = [1, -1, -1]

In [90]:
A[5, -2:] = 1
b[5] = 1
sense[5] = '='

In [91]:
A[6, [2, 3, 4, 7]] = [-1, -1, -1, 1]

In [92]:
A[7, :5] = [450, 600, 750, 200, 150]
b[7] = 600
sense[7] = '>'

In [93]:
obj = np.array([500, 550, 650, 50, 50, 50, 70, 150, 200])

In [94]:
med_school = create_model(A, sense, b, obj, opt=gp.GRB.MINIMIZE, vtype=['B']*n_var)

<ipython-input-94-28483c0021ed>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  med_school = create_model(A, sense, b, obj, opt=gp.GRB.MINIMIZE, vtype=['B']*n_var)


In [95]:
med_school.x

[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]

In [98]:
med_school.objVal # time

770.0

In [99]:
A[7] @ med_school.x # earning

650.0

In [100]:
obj[5]

50

In [101]:
obj[-2]

150

### q2

In [108]:
supply = get_supply(opt_port.x) # supply 
print(supply)
np.sum(supply)

[33.59507662 23.32626652 31.07865661 43.99999987]


131.9999996166892

In [107]:
44+20+50+18

132

In [110]:
x = opt_port.x
x1 = x[0]
x2 = x[1]
x3 = x[2]

e1 = 50*x1
e2 = 50*x2 + 20*x1*x1
e3 = 50*x3 + 20*x2*x2 + 10*x1*x1

In [111]:
e1

38.76402980164214

### q3

In [115]:
A = np.array([[40, 50, 20, 30]])
b = np.array([50])
sense = ['<']
obj = np.array([70, 80, 30, 50])

In [116]:
lb = np.array([0, 0, 0, 0])
ub = np.array([1, 1, 1, 1])

In [117]:
project = create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE, ub=ub, lb=lb)

<ipython-input-117-5b321a618afd>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  project = create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE, ub=ub, lb=lb)


In [122]:
x = project.x
x

[1.0, 0.0, 0.0, 0.3333333333333333]

In [119]:
project.objVal

86.66666666666666

In [123]:
A @ x

array([50.])

In [130]:
A[0, 3] * x[3]

10.0

### q4

8w + 3x + 2y + z =5

w - y + z = 1

w + x + 2z = -4

-2w + 3x - 4y + 13z = 140

In [131]:
A = np.zeros((4, 4))
b = np.zeros(4)
A[0, :] = [8, 3, 2, 1]
A[1, :] = [1, 0, -1, 1]
A[2, :] = [1, 1, 0, 2]
A[3, :] = [-2, 3, -4, 13]

b[0] = 5
b[1] = 1
b[2] = -4
b[3] = 140

In [134]:
x = np.linalg.solve(A, b)
x

array([ -83.5       ,  481.83333333, -285.66666667, -201.16666667])

# Pigskin

In [203]:
M = 50000
capacity = 20000
n_var = 18
n_const = 12
# vars - yi, xi, zi

A = np.zeros((n_const, n_var))
b = np.zeros(n_const)
sense = ['<']*n_const
d = [10000, 15000, 30000, 35000, 25000, 10000]
ub = np.array([1]*6 + [M]*6 + [capacity]*6)
lb = np.array([0]*n_var)
types = ['B']*6 + ['C']*12

In [204]:
i = 0
A[i, [6, 12]] = [1, -1]
b[i] = d[i] - 5000
sense[i] = '='
i += 1

for k in range(1, 6):
    A[i, [6+k, 12+k-1, 12+k]] = [1, 1, -1]
    b[i] = d[i]
    sense[i] = '='
    i += 1

In [205]:
for k in range(6):
    A[i, [k, k+6]] = [-M, 1]
    i += 1

In [206]:
unit_cost = [12.5, 12.55, 12.7, 12.8, 12.85, 12.95]
obj = np.array([50000]*6 + unit_cost + [0.05*c for c in unit_cost])

In [207]:
pigskin = create_model(A, sense, b, obj, opt=gp.GRB.MINIMIZE, vtype=types, ub=ub, lb=lb)

<ipython-input-207-73a4389047bf>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  pigskin = create_model(A, sense, b, obj, opt=gp.GRB.MINIMIZE, vtype=types, ub=ub, lb=lb)


In [208]:
pigskin.objVal

1744550.0

In [209]:
pigskin.x

[1.0,
 -0.0,
 1.0,
 1.0,
 1.0,
 -0.0,
 20000.0,
 0.0,
 30000.0,
 35000.0,
 35000.0,
 0.0,
 15000.0,
 0.0,
 0.0,
 0.0,
 10000.0,
 0.0]

In [210]:
sense

['=', '=', '=', '=', '=', '=', '<', '<', '<', '<', '<', '<']